In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pickle
import logging
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import jax
import jax.numpy as jnp

import arviz as az
import numpyro
from numpyro.diagnostics import hpdi

from hbmep.config import Config
from hbmep.model.utils import Site as site

from core import MixtureModel

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()

logger = logging.getLogger(__name__)


In [2]:
toml_path = "/home/vishu/repos/hbmep-paper/configs/paper/tms/config.toml"
config = Config(toml_path=toml_path)
config.BUILD_DIR = os.path.join(config.BUILD_DIR, "group-comparison")
config.FEATURES = ["participant", "participant_condition"]
config.RESPONSE = ['PKPK_APB', 'PKPK_ECR', 'PKPK_FCR']
config.MCMC_PARAMS["num_warmup"] = 5000
config.MCMC_PARAMS["num_samples"] = 1000
model = MixtureModel(config=config)

src = "/home/vishu/data/hbmep-processed/human/tms/proc_2023-11-28.csv"
df = pd.read_csv(src)
df, encoder_dict = model.load(df=df)


2023-12-14 04:22:12,465 - hbmep.model.baseline - INFO - Initialized mixture_model


2023-12-14 04:22:12,485 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/paper/tms/group-comparison
2023-12-14 04:22:12,486 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/paper/tms/group-comparison
2023-12-14 04:22:12,487 - hbmep.dataset.core - WARNING - Total non-positive observations: 0
2023-12-14 04:22:12,488 - hbmep.dataset.core - WARNING - Total missing observations: 0
2023-12-14 04:22:12,488 - hbmep.dataset.core - INFO - Processing data ...
2023-12-14 04:22:12,490 - hbmep.utils.utils - INFO - func:load took: 0.00 sec


In [3]:
src = "/home/vishu/repos/hbmep-paper/reports/paper/tms/group-comparison/inference.pkl"
with open(src, "rb") as g:
    _, mcmc, posterior_samples = pickle.load(g)


In [4]:
a_mean = posterior_samples["a_mean"]
a_mean.shape

(4000, 2, 3)

In [5]:
((a_mean[:, 0, :] - a_mean[:, 1, :]) > 0).mean(axis=0)

array([0.89675, 0.87   , 0.97025])

In [6]:
model.response

['PKPK_APB', 'PKPK_ECR', 'PKPK_FCR']

In [7]:
response_colors = {
    0: (0.8157, 0.1098, 0.5451),
    1: (0.2055, 0., 0.4000),
    2: (0.3000, 0.3000, 0.3000)
}
prior_color = (.7, .7, .7)


In [8]:
_posterior_samples = posterior_samples.copy()
_posterior_samples["outlier_prob"] = _posterior_samples["outlier_prob"] * 0
prediction_df = model.make_prediction_dataset(df=df, num=5000)
# prediction_df = model.make_prediction_dataset(df=df, num=100, min_intensity=0, max_intensity=82)
posterior_predictive = model.predict(df=prediction_df, posterior_samples=_posterior_samples)


2023-12-14 04:22:13,699 - hbmep.utils.utils - INFO - func:make_prediction_dataset took: 0.02 sec


2023-12-14 04:32:31,255 - hbmep.utils.utils - INFO - func:predict took: 10 min and 17.56 sec


In [9]:
a = posterior_samples[site.a]
a_map = a.mean(axis=0)
mu = posterior_predictive[site.mu]
obs = posterior_predictive[site.obs]
obs_hpdi = hpdi(obs, prob=.95)
a_mean = posterior_samples["a_mean"]


In [11]:
response = ["APB", "ECR", "FCR"]
axis_label_size = 10

""" Plot """
nrows, ncols = 5, 3
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(3 * ncols, 1.8 * nrows), squeeze=False, height_ratios=[1, .5, 1, .5, 1], constrained_layout=True)

rec_ins = []

for muscle_ind in range(model.n_response):
    color = response_colors[muscle_ind]

    for c_counter, c in enumerate([(5, 1), (18, 0)]):
        ind = df[model.features].apply(tuple, axis=1).isin([c])
        temp_df = df[ind].reset_index(drop=True).copy()

        pred_ind = prediction_df[model.features].apply(tuple, axis=1).isin([c])
        temp_pred_df = prediction_df[pred_ind].reset_index(drop=True).copy()

        temp_obs = obs[:, pred_ind, muscle_ind]
        temp_mu = mu[:, pred_ind, muscle_ind]
        temp_obs_hpdi = obs_hpdi[:, pred_ind, muscle_ind]

        ax = axes[2 * c_counter, muscle_ind]
        sns.scatterplot(x=temp_df[model.intensity], y=temp_df[model.response[muscle_ind]], ax=ax, color=color)
        sns.lineplot(x=temp_pred_df[model.intensity], y=temp_mu.mean(axis=0), color=color, ax=ax)
        ax.fill_between(
            temp_pred_df[model.intensity],
            temp_obs_hpdi[0, :],
            temp_obs_hpdi[1, :],
            color=color,
            alpha=.15
        )
        ax.sharex(axes[0, 0])
        ax.sharey(axes[2 * c_counter, 0])

        if (c, muscle_ind) in [((5, 1), 1), ((5, 1), 2)]:
            ins = ax.inset_axes([0.15,0.4,0.8 ,0.5])
            rec_ins.append(ins)
            sns.scatterplot(x=temp_df[model.intensity], y=temp_df[model.response[muscle_ind]], ax=ins, color=color)
            sns.lineplot(x=temp_pred_df[model.intensity], y=temp_mu.mean(axis=0), color=color, ax=ins)
            ins.fill_between(
                temp_pred_df[model.intensity],
                temp_obs_hpdi[0, :],
                temp_obs_hpdi[1, :],
                color=color,
                alpha=.15
            )
            ins.set_xlabel("")
            ins.set_ylabel("")
            ins.xaxis.set_major_locator(plt.MaxNLocator(3))
            ins.yaxis.set_major_locator(plt.MaxNLocator(3))
            # ins.set_xlim(left=4, right=6)
            ins.tick_params(
                axis='both',
                which='both',
                left=True,
                bottom=True,
                right=False,
                top=False,
                labelleft=True,
                labelbottom=True,
                labelright=False,
                labeltop=False,
                labelrotation=15,
                labelsize=8
            )

        ax = axes[2 * c_counter + 1, muscle_ind]
        sns.kdeplot(a[:, *c, muscle_ind], ax=ax, color=color)
        ax.sharex(axes[0, 0])

    ax = axes[4, muscle_ind]
    samples = a_mean[:, 0, muscle_ind] - a_mean[:, 1, muscle_ind]
    # d = dist.Normal(0, 3)
    # prior = d.sample(model.rng_key, (10000, ))
    # sns.kdeplot(prior, ax=ax, color=prior_color, label="Prior")
    sns.kdeplot(samples, ax=ax, color=color, label="Posterior")
    prob = (samples > 0).mean()
    ax.set_title(f"Pr( SCI > Uninjured ) = {prob:.2f}", fontweight="bold", size=8)
    ax.sharex(axes[4, 0])
    ax.sharey(axes[4, 0])
    ax.axvline(x=0, color=prior_color, linestyle="--")

for i in range(nrows):
    for j in range(ncols):
        ax = axes[i, j]
        sides = ['right', 'top']
        for side in sides:
            ax.spines[side].set_visible(False)
        ax.tick_params(
            axis='both',
            which='both',
            left=True,
            bottom=True,
            right=False,
            top=False,
            labelleft=False,
            labelbottom=True,
            labelright=False,
            labeltop=False,
            labelrotation=15
        )
        ax.set_xlabel("")
        ax.set_ylabel("")

for muscle_ind in range(1):
    for c_counter, c in enumerate([(5, 1), (18, 0)]):
        ax = axes[2 * c_counter, muscle_ind]
        ax.set_xlabel("TMS Intensity (%)", size=axis_label_size)

        ax = axes[2 * c_counter + 1, muscle_ind]
        ax.set_xlabel("Threshold (%)", size=axis_label_size)
        ax.tick_params(
            axis='both',
            which='both',
            left=False,
            labelleft=False,
            labelrotation=15
        )

    ax = axes[4, muscle_ind]
    ax.set_xlabel("SCI $-$ Uninjured Threshold (%)", size=axis_label_size)
    ax.tick_params(
        axis='both',
        which='both',
        left=False,
        labelleft=False,
        labelrotation=15
    )

    ax = axes[0, muscle_ind]
    ax.set_title(response[muscle_ind], fontweight="bold")
    # ax.set_title(response[muscle_ind], fontweight="bold", color=response_colors[muscle_ind])

ax = axes[0, 0]
ax.tick_params(
    axis='both',
    which='both',
    labelleft=True,
    labelrotation=15
)
ax.set_ylabel("pk-pk (a.u.)", size=axis_label_size)
ax.set_xticks([0, 40, 80])
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
ax.text(.001, 6.5, "Uninjured", size=10, ha="left", va="center", fontweight="bold")

ax = axes[1, 0]
ax.set_ylabel("Prob. Density", size=axis_label_size)

ax = axes[2, 0]
ax.set_ylabel("pk-pk (a.u.)", size=axis_label_size)
ax.yaxis.set_major_locator(plt.MaxNLocator(3))
ax.tick_params(
    axis='both',
    which='both',
    left=True,
    labelleft=True,
)
ax.text(.001, 0.65, "SCI", size=10, ha="left", va="center", fontweight="bold")

ax = axes[3, 0]
ax.set_ylabel("Prob. Density", size=axis_label_size)

ax = axes[4, 0]
ax.tick_params(
    axis='both',
    which='both',
    left=False,
    labelleft=False,
    labelbottom=True,
    labelrotation=15
)
ax.set_ylabel("Probability Density", size=axis_label_size)
ax.set_xticks([-20, 0, 20, 40])

rec_ins[1].sharex(rec_ins[0])
rec_ins[1].sharey(rec_ins[0])

fig.align_xlabels()
fig.align_ylabels()

dest = os.path.join(model.build_dir, "group-comparison.svg")
fig.savefig(dest, dpi=600)
dest = os.path.join(model.build_dir, "group-comparison.png")
fig.savefig(dest, dpi=600)
logger.info(f"Saved figure to {dest}")
